In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

from keras.layers import Dense, InputLayer, BatchNormalization, Dropout, Conv2D, Flatten, MaxPool2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.io import imread

# LOADING DATA

In [ ]:
dir = "drive/MyDrive/AV/Gender_classification/"

In [ ]:
data = pd.read_csv(dir + 'train.csv')

In [ ]:
data.head()

image_names  class
0   11413.jpg      1
1    9863.jpg      1
2     997.jpg      0
3    5950.jpg      0
4   10454.jpg      1

In [ ]:
images = []
labels = []
i = 0
for img_name in tqdm(data["image_names"]):
    if i % 5 == 0:
      img = imread(dir + 'images/' + img_name, as_gray = True)
      labels.append(data['class'][i])
      images.append(img)
    i += 1

100%|██████████| 12196/12196 [07:47<00:00, 26.09it/s]


In [ ]:
X = np.array(images)

In [ ]:
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [ ]:
y = np.array(labels)

# TRAIN / VAL SPLIT

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 10)

# MODEL

In [ ]:
model = Sequential()
    
model.add(InputLayer(input_shape=(224, 224, 1)))

model.add(Conv2D(64, (3, 3), activation='relu', strides=(1, 1), padding='valid'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='valid'))
model.add(MaxPool2D(pool_size=(4, 4), padding='valid'))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.6))
model.add(Dense(units=512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.6))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 222, 222, 64)      640       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 109, 109, 128)     73856     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 27, 27, 128)       0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 93312)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               23888128  
_________________________________________________________________
batch_normalization_12 (Batc (None, 256)              

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5', monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max')

In [ ]:
model_history = model.fit(X_train, y_train, epochs = 20, batch_size = 128, validation_data=(X_valid, y_valid), callbacks = [checkpoint])

Epoch 1/20
16/16 [==============================] - 6s 321ms/step - loss: 1.0660 - accuracy: 0.5442 - val_loss: 2.9074 - val_accuracy: 0.5164

Epoch 00001: val_accuracy improved from -inf to 0.51639, saving model to best_model.hdf5
Epoch 2/20
16/16 [==============================] - 5s 311ms/step - loss: 0.8274 - accuracy: 0.6332 - val_loss: 0.8420 - val_accuracy: 0.6557

Epoch 00002: val_accuracy improved from 0.51639 to 0.65574, saving model to best_model.hdf5
Epoch 3/20
16/16 [==============================] - 5s 313ms/step - loss: 0.7653 - accuracy: 0.6525 - val_loss: 0.6337 - val_accuracy: 0.7070

Epoch 00003: val_accuracy improved from 0.65574 to 0.70697, saving model to best_model.hdf5
Epoch 4/20
16/16 [==============================] - 5s 315ms/step - loss: 0.7838 - accuracy: 0.6537 - val_loss: 0.5965 - val_accuracy: 0.7234

Epoch 00004: val_accuracy improved from 0.70697 to 0.72336, saving model to best_model.hdf5
Epoch 5/20
16/16 [==============================] - 5s 318ms/st

# EVALUATION

In [ ]:
model.load_weights('best_model.hdf5')

In [ ]:
print('Accuracy on validation set:', accuracy_score(y_valid, model.predict_classes(X_valid)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy on validation set: 0.764344262295082
